In [1]:
import pandas as pd
import string
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku

In [2]:
lego = pd.read_csv('lego_data_clean_translated.csv')
lego.head()


toy_name_en = lego['toy_name_en'].values
print(toy_name_en)

['Himeji Castle' 'New York City' 'London' ... 'Easter Bunny House'
 'Mighty Micros: Supergirl™ vs. Brainiac™'
 'Mighty Micros: Batman™ vs. Harley Quinn™']


In [3]:
def clean_text(txt):
    txt = "".join(t for t in txt if t not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii", 'ignore')
    return txt

toy_name_en_clean = [clean_text(x) for x in toy_name_en]
toy_name_en_clean[:10]

['himeji castle',
 'new york city',
 'london',
 'paris',
 'great pyramid of giza',
 'taj mahal',
 'singapore',
 'statue of liberty',
 'the white house',
 'batcave shadow box']

In [4]:
tokenizer = Tokenizer()
def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    ## convert data to a token sequence
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(toy_name_en_clean)
inp_sequences[:10]

[[697, 8],
 [67, 367],
 [67, 367, 27],
 [58, 698],
 [58, 698, 4],
 [58, 698, 4, 699],
 [700, 701],
 [703, 4],
 [703, 4, 369],
 [2, 170]]

In [5]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

print(predictors)
print(label)
print(max_sequence_len)

[[  0   0   0 ...   0   0 697]
 [  0   0   0 ...   0   0  67]
 [  0   0   0 ...   0  67 367]
 ...
 [  0   0   0 ... 216 696  23]
 [  0   0   0 ... 696  23  19]
 [  0   0   0 ...  23  19 241]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
10


In [6]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1  # Length of input sequences
    model = Sequential()
    model.add(Embedding(total_words, 10, input_length=input_len))  # Input layer
    model.add(LSTM(100))  # LSTM layer with 100 units
    model.add(Dropout(0.1))  # Dropout for regularization
    model.add(Dense(total_words, activation='softmax'))  # Output layer
    model.compile(loss='categorical_crossentropy', optimizer='adam')  # Compile the model
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

/Users/nico/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [7]:
model.fit(predictors, label, epochs=250, verbose=1)

Epoch 1/250
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 7.1537
Epoch 2/250
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.4210
Epoch 3/250
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.3539
Epoch 4/250
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.3024
Epoch 5/250
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.2526
Epoch 6/250
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.1881
Epoch 7/250
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.1472
Epoch 8/250
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.0506
Epoch 9/250
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.9925
Epoch 10/250
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.8493
Epoch 11/250
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.8040
Epoch 12/250
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.6751
Epoch 13/250
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.6485
Epoch 14/250
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.5182
Epoch 15/250
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.3762
Epoc

In [8]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import random

def generate_text(seed_text, next_words, model, max_sequence_len, tokenizer):
    # If the seed_text is empty, randomly pick a word from the tokenizer's word index
    if seed_text == "":
        seed_text = random.choice(list(tokenizer.word_index.keys()))

    seed_text += " " + random.choice(list(tokenizer.word_index.keys()))

    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted = np.argmax(predicted, axis=-1)  # Get the index of the maximum prediction

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text.title()

In [10]:
# Example usage with an empty string as seed_text
print(generate_text("Batman:", 3, model, max_sequence_len, tokenizer))

Batman: Razor Brick Keychain Hill
